[Reference](https://medium.com/@ccpythonprogramming/dynamically-import-csv-files-into-sqlite-with-python-3c9ba07fe559)

In [1]:
import sqlite3
import pandas as pd

def create_table(c, table_name, columns):
    columns_definition = ', '.join([f"{col} TEXT" for col in columns])
    create_statement = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_definition})"
    c.execute(create_statement)

def insert_into_db(table_name, df):
    conn = sqlite3.connect('example.db')
    c = conn.cursor()

    # Create table based on DataFrame columns
    create_table(c, table_name, df.columns)

    # Insert data into the table
    for _, row in df.iterrows():
        c.execute(f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['?'] * len(df.columns))})", tuple(row))

    conn.commit()
    conn.close()
    print(f"Data from {table_name} has been inserted into the database.")

def process_excel(file_path, sheet_name=None):
    # Read Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    table_name = sheet_name if sheet_name else 'excel_data'
    insert_into_db(table_name, df)

def process_csv(file_path):
    # Read CSV file
    df = pd.read_csv(file_path)
    table_name = file_path.split('/')[-1].replace('.csv', '')
    insert_into_db(table_name, df)

# Example usage
# Processing Excel file with multiple sheets
excel_file_path = 'data.xlsx'
sheet_names = pd.ExcelFile(excel_file_path).sheet_names

for sheet in sheet_names:
    process_excel(excel_file_path, sheet_name=sheet)

# Processing a CSV file
csv_file_path = 'data.csv'
process_csv(csv_file_path)